# Limpieza de Datos

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

#### 6. Normalizar/Establecer Rangos
- Si es necesario, normaliza o escala variables numéricas.
- Asegúrate de que los valores estén dentro de rangos lógicos.

#### 7. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.

In [2]:
import pandas as pd  # Data manipulation
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Data visualization
import seaborn as sns  # Data visualization
import plotly.express as px  # Interactive data visualization

import sys
sys.path.append('..')  # Ajusta el path si es necesario

from main import cargar_datos, guardar_datos

In [3]:
# Cargar datos
datos = cargar_datos()

In [4]:
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


## Desarrollo

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

In [7]:
# Ver tipos de datos
datos.dtypes

Order ID             object
Date                 object
Product              object
Category             object
Price                 int64
Quantity              int64
Total Sales           int64
Customer Name        object
Customer Location    object
Payment Method       object
Status               object
dtype: object

**Interpretación**:

* Debemos cambiar el tipo de dato de *Date* al correcto

In [9]:
# Cambiar tipo de dato de date
datos['Date'] = pd.to_datetime(datos['Date'], format='%d-%m-%y')
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,2025-03-14,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,2025-03-20,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,2025-02-15,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,2025-02-19,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,2025-03-10,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

In [10]:
datos.isnull().sum()

Order ID             0
Date                 0
Product              0
Category             0
Price                0
Quantity             0
Total Sales          0
Customer Name        0
Customer Location    0
Payment Method       0
Status               0
dtype: int64

* *No hay NULOS*

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

In [14]:
duplicados = datos.duplicated().sum()
duplicados

if duplicados == 0:
    print("No hay duplicados en el DataFrame.")
else:
    print(f"Hay {duplicados} duplicados en el DataFrame.")
    datos = datos.drop_duplicates()
    print("Duplicados eliminados.")

No hay duplicados en el DataFrame.


#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

In [ ]:
# Unificar formatos de texto par

#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

#### 6. Normalizar/Establecer Rangos
- Si es necesario, normaliza o escala variables numéricas.
- Asegúrate de que los valores estén dentro de rangos lógicos.

#### 7. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.